# Try

we can try & experiment here to combine different models here..

predictions = [list(set(a + b + c + d)) for a, b, c, d in zip(predictions1, predictions2,predictions3, predictions4)]

 

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [ ]:
%%capture

# Dirty code to make it work

import sys
!cp -r ../input/openai-clip/CLIP/CLIP-main /tmp/

# Kaggle likes to unpack .gz files in datasets... so we have to pack it back
!gzip -c /tmp/CLIP-main/clip/bpe_simple_vocab_16e6.txt > /tmp/CLIP-main/clip/bpe_simple_vocab_16e6.txt.gz
sys.path.append('/tmp/CLIP-main')

!pip install ../input/openai-clip/ftfy-5.9/ftfy-5.9 \
             ../input/openai-clip/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl \
             ../input/openai-clip/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl \
             ../input/faiss-163/faiss_gpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl

In [ ]:
import numpy as np 
import pandas as pd 

import math
import random 
import os 
import cv2
import timm

from tqdm import tqdm 

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import torch 
from torch.utils.data import Dataset 
from torch import nn
import torch.nn.functional as F 

import gc
import cudf
import cuml
import cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

In [ ]:
class CFG:
    
    img_size = 512
    fc_dim = 512
    batch_size = 12
    seed = 2020
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    classes = 11014
    
    model_name = 'tf_efficientnet_b5_ns'
#     model_name1 =  'tf_efficientnet_b4'
    model_name2 = 'eca_nfnet_l0'
    model_name3 = 'efficientnet_b3'
    
    model_path = '../input/shopee-pytorch-models/arcface_512x512_eff_b5_.pt'
#     model_path1 = '../input/utils-shopee/arcface_512x512_tf_efficientnet_b4_LR.pt'
    model_path2 = '../input/shopee-pytorch-models/arcface_512x512_nfnet_l0 (mish).pt'
    model_path3 = '../input/arcfacebin/arcface04.bin'
    
    scale = 30 
    margin = 0.5

In [ ]:
def read_dataset():
    df = pd.read_csv('../input/shopee-product-matching/test.csv')
    df_cu = cudf.DataFrame(df)
    image_paths = '../input/shopee-product-matching/test_images/' + df['image']
    return df, df_cu, image_paths

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)

In [ ]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions'], row['text_predictions'], row['phash'], row['pred_matches02']])
    return ' '.join( np.unique(x))

In [ ]:
def get_image_predictions(df, embeddings1, embeddings3, embeddings4, threshold = 3.4):
    
    if len(df) > 3:
        KNN = 50
    else : 
        KNN = 3
    
    #--01
    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings1)
    distances, indices = model.kneighbors(embeddings1)
    
    threshold = 1.7
    predictions1 = []
    for k in tqdm(range(embeddings1.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = list(df['posting_id'].iloc[ids])
        predictions1.append(posting_ids)
        
    del model, distances, indices, embeddings1
    gc.collect()

#     #--02
#     model = NearestNeighbors(n_neighbors = KNN)
#     model.fit(embeddings2)
#     distances, indices = model.kneighbors(embeddings2)
    
#     threshold = 4.5
#     predictions2 = []
#     for k in tqdm(range(embeddings2.shape[0])):
#         idx = np.where(distances[k,] < threshold)[0]
#         ids = indices[k,idx]
#         posting_ids = list(df['posting_id'].iloc[ids])
#         predictions2.append(posting_ids)
        
#     del model, distances, indices, embeddings2
#     gc.collect()
    
    #--03
    model = NearestNeighbors(n_neighbors = KNN, metric = 'cosine')
    model.fit(embeddings3)
    distances, indices = model.kneighbors(embeddings3)
    
    threshold=0.36
    predictions3 = []
    for k in tqdm(range(embeddings3.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = list(df['posting_id'].iloc[ids])
        predictions3.append(posting_ids)
        
    del model, distances, indices, embeddings3
    gc.collect()
    
    #--04 ################################################ new ###############################
    model = NearestNeighbors(n_neighbors = KNN)
    model.fit(embeddings4)
    distances, indices = model.kneighbors(embeddings4)
    
#     threshold=2.9
    predictions4 = []
    for k in tqdm(range(embeddings4.shape[0])):
        idx = np.where(distances[k,] < 2.9)[0]
        ids = indices[k,idx]
        posting_ids = list(df['posting_id'].iloc[ids])
        predictions4.append(posting_ids)
        
    del model, distances, indices, embeddings4
    gc.collect()
    # combine predictions(i.e. image IDs) of all the models & remove the duplicates.
    
    
    # we can try & experiment here to combine different models here..
#     predictions = [list(set(a + c)) for a, b, c, d in zip(predictions1, predictions2, predictions3, predictions4)]
    predictions = [list(a + c + d) for a, c, d in zip(predictions1, predictions3, predictions4)]
    return predictions

In [ ]:
def get_test_transforms():

    return A.Compose(
        [
            A.Resize(CFG.img_size,CFG.img_size,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

In [ ]:
class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin
        
    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = CFG.model_name,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = False,
        pretrained = False):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'nfnet_f3':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x
    
    
# class ShopeeModel1(nn.Module):
#     def __init__(
#         self,
#         n_classes = CFG.classes,
#         model_name = CFG.model_name1,
#         fc_dim = CFG.fc_dim,
#         margin = CFG.margin,
#         scale = CFG.scale,
#         use_fc = True,
#         pretrained = True):

#         super(ShopeeModel1, self).__init__()
#         print('Building Model Backbone for {} model'.format(model_name))

#         self.backbone = timm.create_model(model_name, pretrained=pretrained)
#         in_features = self.backbone.classifier.in_features
#         self.backbone.classifier = nn.Identity()
#         self.backbone.global_pool = nn.Identity()
#         self.pooling =  nn.AdaptiveAvgPool2d(1)
#         self.use_fc = use_fc

#         if use_fc:
#             self.dropout = nn.Dropout(p=0.1)
#             self.classifier = nn.Linear(in_features, fc_dim)
#             self.bn = nn.BatchNorm1d(fc_dim)
#             self._init_params()
#             in_features = fc_dim

#         self.final = ArcMarginProduct(
#             in_features,
#             n_classes,
#             scale = scale,
#             margin = margin,
#             easy_margin = False,
#             ls_eps = 0.0
#         )

#     def _init_params(self):
#         nn.init.xavier_normal_(self.classifier.weight)
#         nn.init.constant_(self.classifier.bias, 0)
#         nn.init.constant_(self.bn.weight, 1)
#         nn.init.constant_(self.bn.bias, 0)

#     def forward(self, image, label):
#         features = self.extract_features(image)
#         if self.training:
#             logits = self.final(features, label)
#             return logits
#         else:
#             return features

#     def extract_features(self, x):
#         batch_size = x.shape[0]
#         x = self.backbone(x)
#         x = self.pooling(x).view(batch_size, -1)

#         if self.use_fc and self.training:
#             x = self.dropout(x)
#             x = self.classifier(x)
#             x = self.bn(x)
#         return x
    
class ShopeeModel2(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = CFG.model_name2,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel2,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'efficientnet_b3':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif model_name == 'tf_efficientnet_b5_ns':
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x
################################ new ##################################
class ShopeeModel3(nn.Module):

    def __init__(self,
                 n_classes = 10873,  #<<<<<<<______________________________corrected classes
                 model_name=CFG.model_name3,
                 use_fc=False,
                 fc_dim=512,
                 dropout=0.0,
                 loss_module='softmax',#<<<<<<<______________________________
                 s=30.0,
                 margin=0.50,
                 ls_eps=0.0,
                 theta_zero=0.785,
                 pretrained=False):
        """
        :param n_classes:
        :param model_name: name of model from pretrainedmodels
            e.g. resnet50, resnext101_32x4d, pnasnet5large
        :param pooling: One of ('SPoC', 'MAC', 'RMAC', 'GeM', 'Rpool', 'Flatten', 'CompactBilinearPooling')
        :param loss_module: One of ('arcface', 'cosface', 'softmax')
        """
        super(ShopeeModel3, self).__init__()
        print('Model building for {} backbone'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)
        final_in_features = self.backbone.classifier.in_features
        
        self.backbone.classifier = nn.Identity()
        self.backbone.global_pool = nn.Identity()
        
        self.pooling =  nn.AdaptiveAvgPool2d(1)
            
        self.use_fc = use_fc
        if use_fc:
            self.dropout = nn.Dropout(p=dropout)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim

        self.loss_module = loss_module
        if loss_module == 'arcface':
            self.final = ArcMarginProduct(final_in_features, n_classes,
                                          scale=s, margin=margin, easy_margin=False, ls_eps=ls_eps)
        elif loss_module == 'cosface':
            self.final = AddMarginProduct(final_in_features, n_classes, scale=s, margin=margin)
        elif loss_module == 'adacos':
            self.final = AdaCos(final_in_features, n_classes, margin=margin, theta_zero=theta_zero)
        else:
            self.final = nn.Linear(final_in_features, n_classes)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x, label):
        feature = self.extract_feat(x)
        if self.loss_module in ('arcface', 'cosface', 'adacos'):
            logits = self.final(feature, label)
        else:
            logits = self.final(feature)
        return feature,logits

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)

        return x

In [ ]:
# https://www.kaggle.com/parthdhameliya77/pytorch-eca-nfnet-l0-image-tfidf-inference
class Mish_func(torch.autograd.Function):
    
    """from: https://github.com/tyunist/memory_efficient_mish_swish/blob/master/mish.py"""
    
    @staticmethod
    def forward(ctx, i):
        result = i * torch.tanh(F.softplus(i))
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
  
        v = 1. + i.exp()
        h = v.log() 
        grad_gh = 1./h.cosh().pow_(2) 

        # Note that grad_hv * grad_vx = sigmoid(x)
        #grad_hv = 1./v  
        #grad_vx = i.exp()
        
        grad_hx = i.sigmoid()

        grad_gx = grad_gh *  grad_hx #grad_hv * grad_vx 
        
        grad_f =  torch.tanh(F.softplus(i)) + i * grad_gx 
        
        return grad_output * grad_f 


class Mish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        pass
    def forward(self, input_tensor):
        return Mish_func.apply(input_tensor)


def replace_activations(model, existing_layer, new_layer):
    
    """A function for replacing existing activation layers"""
    
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            model._modules[name] = replace_activations(module, existing_layer, new_layer)

        if type(module) == existing_layer:
            layer_old = module
            layer_new = new_layer
            model._modules[name] = layer_new
    return model

In [ ]:
def get_image_embeddings(image_paths, model_name = CFG.model_name):
    
    #---00
    embeds = []
    
    model = ShopeeModel(model_name = model_name)
    model.eval()
    model.load_state_dict(torch.load(CFG.model_path))
    model = model.to(CFG.device)

    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    del model, image_embeddings
    image_embeddings1 = np.concatenate(embeds)
#     image_embeddings1 = normalize(image_embeddings1)
    print(f'image embeddings1 shape is {image_embeddings1.shape}')
    del embeds
    gc.collect()
    
#     #---01
#     model = ShopeeModel1(pretrained=False).to(CFG.device)
#     model.load_state_dict(torch.load(CFG.model_path1))
#     model.eval()

#     image_dataset = ShopeeDataset(image_paths=image_paths, transforms=get_test_transforms())
#     image_loader = torch.utils.data.DataLoader(
#         image_dataset,
#         batch_size=CFG.batch_size,
#         num_workers=4
#     )

#     embeds1 = []
#     with torch.no_grad():
#         for img,label in tqdm(image_loader): 
#             img = img.cuda()
#             label = label.cuda()
#             features = model(img,label)
#             image_embeddings = features.detach().cpu().numpy()
#             embeds1.append(image_embeddings)

#     del model, image_embeddings
#     image_embeddings2 = np.concatenate(embeds1)
#     print(f'image embeddings2 shape is {image_embeddings2.shape}')
#     del embeds1
#     gc.collect()
    
    #---02
    model = ShopeeModel2()
    model.eval()
    model = replace_activations(model, torch.nn.SiLU, Mish())

    model.load_state_dict(torch.load(CFG.model_path2))
    model = model.to(CFG.device)
    
    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    embeds2 = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds2.append(image_embeddings)
    del model
    image_embeddings3 = np.concatenate(embeds2)
#     image_embeddings3 = normalize(image_embeddings3)

    print(f'image embeddings3 shape is {image_embeddings3.shape}')
    del embeds2
    gc.collect()
    
    #---03 ################################ new ####################################   
    embeds3 = []
    model = ShopeeModel3()
    model.eval()
    model.load_state_dict(torch.load(CFG.model_path3),strict=False)
    model = model.to(CFG.device)

    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat, _ = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds3.append(image_embeddings)
    
    del model
    image_embeddings4 = np.concatenate(embeds3)
#     image_embeddings4 = normalize(image_embeddings4)
    print(f'Our image embeddings4 shape is {image_embeddings4.shape}')
    del embeds3
    gc.collect()
    
    
    
    #---
    
    # Method 1 of ensembling
    # image_embeddings = np.concatenate([image_embeddings1, image_embeddings2, image_embeddings3], 1)
    
    return image_embeddings1, image_embeddings3, image_embeddings4

In [ ]:
#https://www.kaggle.com/cdeotte/part-2-rapids-tfidfvectorizer-cv-0-700#Use-Text-Embeddings

def get_text_predictions(df, max_features = 25_000):
    
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(df)//CHUNK
    if len(df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.75)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o.tolist())
    
    del model,text_embeddings
    gc.collect()
    return preds

In [ ]:
df,df_cu,image_paths = read_dataset()
df.head()

In [ ]:
image_embeddings1, image_embeddings3, image_embeddings4 = get_image_embeddings(image_paths.values)

In [ ]:
image_predictions = get_image_predictions(df, image_embeddings1, image_embeddings3, image_embeddings4, threshold = 1.7)
text_predictions = get_text_predictions(df, max_features = 25_000)

# image predictinon & text_prediction

In [ ]:
df['image_predictions'] = image_predictions
df['text_predictions'] = text_predictions

# phash

In [ ]:
phash = df.groupby('image_phash').posting_id.agg('unique').to_dict()
df['phash'] = df.image_phash.map(phash)
df['phash'] = df.phash.apply(lambda x: x.tolist())

In [ ]:
df_test = df.copy()
df.head(3)

In [ ]:
df = df[['posting_id', 'image_predictions', 'text_predictions', 'phash']]
df.head(2)

# #################################################################################################

# clipAI

In [ ]:
df_test.set_index('posting_id', inplace=True)

In [ ]:
df_test.head(3)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import clip
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import re
from clip.simple_tokenizer import SimpleTokenizer
import faiss
import matplotlib.pyplot as plt
# from triplet_loss import TripletLoss

import warnings
warnings.filterwarnings('ignore')

In [ ]:
_tokenizer = SimpleTokenizer()

# Copied from https://github.com/openai/CLIP/blob/beba48f35392a73c6c47ae67ddffced81ad1916d/clip/clip.py#L164
# but with relaxed exception
def tokenize(texts, context_length: int = 77) -> torch.LongTensor:
    if isinstance(texts, str):
        texts = [texts]

    sot_token = _tokenizer.encoder["<|startoftext|>"]
    eot_token = _tokenizer.encoder["<|endoftext|>"]
    all_tokens = [[sot_token] + _tokenizer.encode(text) + [eot_token] for text in texts]
    result = torch.zeros(len(all_tokens), context_length, dtype=torch.long)

    for i, tokens in enumerate(all_tokens):
        n = min(len(tokens), context_length)
        result[i, :n] = torch.tensor(tokens)[:n]
        if len(tokens) > context_length:
            result[i, -1] = tokens[-1]

    return result

In [ ]:
def l2_normalize(features):
    return features / features.norm(2, dim=1, keepdim=True)

In [ ]:
# Remove EMOJI
RE_EMOJI = re.compile(r"\\x[A-Za-z0-9./]+", flags=re.UNICODE)

def strip_emoji(text):
    return RE_EMOJI.sub(r'', text)

In [ ]:
class RollingMean():
    def __init__(self):
        self.n = 0
        self.mean = 0
        
    def update(self, value):
        self.mean = (self.mean * self.n + value) / (self.n+1)
        self.n += 1
        
    def result(self):
        return self.mean

In [ ]:
class SameGroupSampler(Sampler):
    def __init__(self, df ,ds):
        super().__init__(ds)
        
        # Create a dictionary of posting_id -> index in dataset
        self.index_to_position = dict(zip(df.index, range(len(df))))
        
        # Create a Series of label_group -> set(posting_id)
        self.label_group = df.reset_index().groupby('label_group')['posting_id'].apply(set).map(sorted).map(np.array)

    def __len__(self):
        return len(self.label_group)
        
    def __iter__(self):
        for _ in range(len(self)):
            # Sample one label_group
            label_group_sample = self.label_group.sample(1).iloc[0]
            
            # Sample two posting_id's
            sample1, sample2 = np.random.choice(label_group_sample, 2, replace=False)
            
            yield self.index_to_position[sample1]
            yield self.index_to_position[sample2] 

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, images_path, preprocess):
        super().__init__()
        self.df = df
        self.images_path = images_path
        self.preprocess = preprocess
        self.has_target = ('label_group' in df)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        image = self.preprocess(Image.open(self.images_path / row['image']))
        text = tokenize([strip_emoji(row['title'])])[0]
        
        if self.has_target:
            return image, text, row['label_group']
        else:
            return image, text, 0

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class CLIPClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.clip, self.preprocess = clip.load("../input/openai-clip/ViT-B-32.pt", device='cpu', jit=False)
        
        self.embed_dim = self.clip.text_projection.shape[1]
    
        self.classification_head = nn.Linear(self.embed_dim, n_classes)
    
    def forward(self, images, texts, return_classification=False):
        images_features = self.clip.encode_image(images)
        texts_features = self.clip.encode_text(texts)

        # Average images and text features, because CLIP was trained to align them
        features = l2_normalize(l2_normalize(images_features) + l2_normalize(texts_features))

        if return_classification:
            classification_output = self.classification_head(features)
            
            return features, classification_output
        else:
            return features

In [ ]:
def find_similarities_and_indexes(df, images_path, top_n=100, features_file=None):
    # Create pytorch Dataset/DataLoader
    ds = MyDataset(df, images_path, model.preprocess)
    dl = DataLoader(ds, batch_size=32, shuffle=False, num_workers=2)

    # Allocate memory for features
    features = np.empty((len(df), model.embed_dim), dtype=np.float32)

    # Begin predict
    i = 0
    for images, texts, _ in tqdm(dl):
        n = len(images)
        with torch.no_grad():
            # Generate image and text features
            batch_features = model(images.to(device), texts.to(device), return_classification=False)

        # Average images and text features, because CLIP was trained to align them
        features[i:i+n] = batch_features.cpu()

        i += n

    # Option to save these features (may be usefull to tune cut value)
    if features_file is not None:
        np.save(features_file, features)

    # Create index
    index = faiss.IndexFlatIP(features.shape[1])
    index.add(features)

    # Search index
    return index.search(features, top_n)

    # TODO: try range_search
    # lims, similarities, indexes = index_test.range_search(test_features, GROUP_CUT)

In [ ]:
n_classes = 10875

In [ ]:
model = CLIPClassifier(n_classes).to(device)
model.load_state_dict(torch.load('../input/arcfacebin/clip_image_text_4dot66.bin'))

## run test set

In [ ]:

GROUP_CUT = 0.73  # Use option `RUN_ON_TRAIN` to find this number. It will be desabled at submission time
test_images_path = Path('../input/shopee-product-matching/test_images')
# Find similar matches
similarities, indexes = find_similarities_and_indexes(df_test, test_images_path)

In [ ]:
# Apply cutoff of similiarites
test_are_same_groups = (similarities > GROUP_CUT)

In [ ]:
# Build submission
results = []

for i, (test_is_same_group, index_result) in enumerate(zip(test_are_same_groups, indexes)):
    row_results = set(df_test.index[index_result[test_is_same_group]])
    
    results.append({
        'posting_id': df_test.index[i],
#         'pred_matches02': ' '.join(row_results),
        'pred_matches02': list(row_results)
    })
    
df_sub = pd.DataFrame(results)

In [ ]:
result = pd.merge(df, df_sub, on='posting_id')

In [ ]:
# result['matches'] = result.apply(combine_predictions, axis = 1)
# result.head()

In [ ]:
result['tot'] = result.image_predictions+result.text_predictions+result.phash+result.pred_matches02

In [ ]:
import collections
def result_on_count(row, top_k=1):
#     res = []
#     count_di = collections.Counter(row)
#     for key, value in count_di.items():
#         if value >= top_k:
#             res.append(key)
#     return sorted(count_di, key=lambda x: count_di[x], reverse=True)[:top_k]
    count = pd.value_counts(row)
    return ' '.join(count[count>=top_k].index)

In [ ]:
# def combine_predictions(row):
#     x = np.concatenate([row['matches']])
#     return ' '.join( np.unique(x))

In [ ]:
result['matches'] = result.tot.apply(lambda x: result_on_count(x, 2))
# result['matches'] = result.apply(combine_predictions, axis=1)

In [ ]:
result.head(3)

# results

In [ ]:
result[['posting_id', 'matches']].to_csv('submission.csv', index = False)